In [1]:
import numpy as np
import pandas as pd
import nltk

In [2]:
data=pd.read_csv("data.csv")

In [3]:
data.head()

,Rating,Title,Date,Verified Purchase,Body,Helpful Votes
0,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,Dummy phone !!!,10-May-18,Yes,I recieved a display dummy phone this was a ma...,82.0
2,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,Not a New iPhone Beware,20-Jun-18,Yes,It wasn't new. The serial number in the phone...,17.0
4,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
data.dropna(inplace=True)

In [5]:
data['title_body']=data['Title']+ ' ' + data['Body']

In [6]:
data.reset_index(inplace=True)
data.drop(['index','Title','Body','Date','Verified Purchase','Helpful Votes'],axis=1,inplace=True)

In [56]:
data[data.Rating==3]

,Rating,title_body
8,3.0,Don't waste your money Overpriced for what you...
85,3.0,Three Stars It was good


In [8]:
inp,out = np.array(data['title_body']),np.array(data['Rating'])

In [9]:
import string
from nltk.corpus import stopwords
stop = stopwords.words('english') + ['phone','iphone','x','apple','samsung']
for i in range(len(inp)):
    new_doc=''
    for j in range(len(inp[i])):
        if not inp[i][j] in string.punctuation:
            new_doc+=(inp[i][j])
        else:
            new_doc+=' '
    new_doc.strip()
    inp[i]=new_doc.strip()

In [10]:
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize

In [11]:
lem=WordNetLemmatizer()

def get_simple_tag(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def NotNumber(w):
    for ch in w:
        if(ch>='0' and ch<='9'):
            return False
    return True
    
def Clean_Words(doc):
    doc=word_tokenize(doc)
    doc = [word for word in doc if word.lower() not in stop and NotNumber(word)==True]
    pos = pos_tag(doc)
    cleaned_words = [lem.lemmatize(w,pos=get_simple_tag(t)).lower() for w,t in pos]
    string=""
    for w in cleaned_words:
        string+=w
        string+=' '
    return string.strip()

In [12]:
clean_in=np.array([Clean_Words(doc) for doc in inp])
sentiment = ['negative','negative','negative','neutral','positive','positive']
clean_out=np.array([sentiment[int(rating)] for rating in out])

In [13]:
np.array([len(doc.split()) for doc in clean_in]).sum(),np.array([len(doc.split()) for doc in clean_in]).mean()

(2726, 23.5)

In [14]:
np.savetxt('Word Clouds/X_neg_clean_reviews.csv',np.array(['text']+[clean_in[i] for i in range(len(clean_in)) if clean_out[i]=='negative']),fmt='%s',delimiter=',')
np.savetxt('Word Clouds/X_neut_clean_reviews.csv',np.array(['text']+[clean_in[i] for i in range(len(clean_in)) if clean_out[i]=='neutral']),fmt='%s',delimiter=',')
np.savetxt('Word Clouds/X_pos_clean_reviews.csv',np.array(['text']+[clean_in[i] for i in range(len(clean_in)) if clean_out[i]=='positive']),fmt='%s',delimiter=',')

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
x_train,x_test,y_train,y_test=train_test_split(clean_in,clean_out)

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [18]:
vec=TfidfVectorizer(max_features=3000,max_df=0.8,min_df=0.1,ngram_range=(1,3),)

In [19]:
vec.fit(x_train)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.8, max_features=3000, min_df=0.1,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [20]:
x_train=vec.transform(x_train)
x_test=vec.transform(x_test)

In [21]:
from sklearn.naive_bayes import MultinomialNB,GaussianNB, BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

In [22]:
clf1 = MultinomialNB()

In [23]:
clf1.fit(x_train,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [24]:
y_pred=clf1.predict(x_test)

In [25]:
from sklearn.metrics import classification_report

In [26]:
print(classification_report(y_test,y_pred))
print('Accuracy = ', clf1.score(x_test,y_test)*100,'%')

             precision    recall  f1-score   support

   negative       0.67      0.73      0.70        11
   positive       0.82      0.78      0.80        18

avg / total       0.76      0.76      0.76        29

Accuracy =  75.86206896551724 %


In [27]:
clf2=BernoulliNB()
clf2.fit(x_train.todense(),y_train)
clf2.score(x_test.todense(),y_test)

0.8275862068965517

In [28]:
y_pred = clf2.predict(x_test.todense())

In [29]:
print(classification_report(y_test,y_pred))
print('Accuracy = ', clf2.score(x_test.todense(),y_test)*100,'%')

             precision    recall  f1-score   support

   negative       0.73      1.00      0.85        11
    neutral       0.00      0.00      0.00         0
   positive       1.00      0.72      0.84        18

avg / total       0.90      0.83      0.84        29

Accuracy =  82.75862068965517 %


C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [30]:
clf3 = LogisticRegression()
clf3.fit(x_train,y_train)
clf3.score(x_test,y_test)

0.7586206896551724

In [31]:
y_pred = clf3.predict(x_test)

In [32]:
print(classification_report(y_test,y_pred))
print('Accuracy = ', clf3.score(x_test.todense(),y_test)*100,'%')

             precision    recall  f1-score   support

   negative       0.70      0.64      0.67        11
   positive       0.79      0.83      0.81        18

avg / total       0.76      0.76      0.76        29

Accuracy =  75.86206896551724 %


In [33]:
clf4 = RandomForestClassifier()
clf4.fit(x_train,y_train)
clf4.score(x_test,y_test)

0.6896551724137931

In [34]:
y_pred=clf4.predict(x_test)

In [35]:
print(classification_report(y_test,y_pred))
print('Accuracy = ', clf4.score(x_test.todense(),y_test)*100,'%')

             precision    recall  f1-score   support

   negative       0.57      0.73      0.64        11
   positive       0.80      0.67      0.73        18

avg / total       0.71      0.69      0.69        29

Accuracy =  68.96551724137932 %


In [36]:
clf5 = SVC()
clf5.fit(x_train,y_train)
clf5.score(x_test,y_test)

0.6206896551724138

In [37]:
y_pred=clf5.predict(x_test)

In [38]:
print(classification_report(y_test,y_pred))
print('Accuracy = ', clf5.score(x_test.todense(),y_test)*100,'%')

             precision    recall  f1-score   support

   negative       0.00      0.00      0.00        11
   positive       0.62      1.00      0.77        18

avg / total       0.39      0.62      0.48        29

Accuracy =  62.06896551724138 %


C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [57]:
test = 'Three stars'
test_final = Clean_Words(test)
test_data=[]
test_data.append(test_final)
f_t_d=vec.transform(test_data)
clf2.predict(f_t_d)

array(['negative'], dtype='<U8')